In [ ]:
!pip install pybtex
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 KB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 3.0 MB/s eta 0:00:00
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.2.0
    Uninstalling xlrd-1.2.0:
      Successfully uninstalled xlrd-1.2.0


In [ ]:
import pandas as pd
import numpy as np
import json, requests
from bs4 import BeautifulSoup
from multiprocessing import  Pool
#from pybtex.database.input import bibtex

In [ ]:
drive_base = "/content/drive/MyDrive/Colab Notebooks/RSL/" 

In [ ]:
df_springer_Chapter1 = pd.read_csv(f"{drive_base}BibTeX/Springer Link/csv2/Chapter1.csv")
df_springer_Chapter2 = pd.read_csv(f"{drive_base}BibTeX/Springer Link/csv2/Chapter2.csv")
df_springer_Paper = pd.read_csv(f"{drive_base}BibTeX/Springer Link/csv2/Conference Paper.csv")
df_springer_article = pd.read_csv(f"{drive_base}BibTeX/Springer Link/csv2/article.csv")



In [ ]:
df_springer = pd.concat([df_springer_Chapter1, df_springer_Chapter2, df_springer_Paper, df_springer_article])

In [ ]:

len(df_springer.index)

2928

In [ ]:
df_springer.columns.values

array(['Item Title', 'Publication Title', 'Book Series Title',
       'Journal Volume', 'Journal Issue', 'Item DOI', 'Authors',
       'Publication Year', 'URL', 'Content Type'], dtype=object)

In [ ]:
def getAbstractByURLSpring(url):
  #url = "https://link.springer.com/article/" + itemDOI

  res = requests.get(url)

  soup = BeautifulSoup(res.text, 'html')

  abstract = soup.find('div', {"id":"Abs1-content"})

  if (abstract == None):
    print("Abstract not found: " + url)
    return np.nan

  firstFind = 1
  for p_tag in abstract.find_all('h3'):
    if firstFind:
      p_tag.string = p_tag.string + "\n"
      firstFind = 0
    else:
      p_tag.string = "\n" + p_tag.string + "\n"

  keywords = soup.findAll('li', {"class": "c-article-subject-list__subject"})

  keys = "; ".join(map(str, [keyword.text for keyword in keywords]))


  return [abstract.text, keys]

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
  df_split = np.array_split(df, n_cores)
  pool = Pool(n_cores)
  df = pd.concat(pool.map(func, df_split))
  pool.close()
  pool.join()
  return df

In [ ]:
def add_features(df):
  df[['Abstract','Keywords']]  = df.apply(lambda row: getAbstractByURLSpring(row.URL), axis = 1, result_type='expand')

  return df

Executa função no método normal:

In [ ]:
df_springer1 = add_features(df_springer)

Executa função no método de paralelismo:

In [ ]:
df_springer2 = parallelize_dataframe(df_springer, add_features, 8)


Abstract not found: http://link.springer.com/article/10.1007/s11600-022-00960-y
Abstract not found: http://link.springer.com/article/10.1007/s10346-021-01821-3
Abstract not found: http://link.springer.com/chapter/10.1007/978-3-319-91146-5_1
Abstract not found: http://link.springer.com/article/10.1186/s13756-021-00974-z
Abstract not found: http://link.springer.com/article/10.1007/s40860-020-00110-5
Abstract not found: http://link.springer.com/article/10.1007/s11307-021-01691-0
Abstract not found: http://link.springer.com/article/10.1007/s42524-021-0185-x
Abstract not found: http://link.springer.com/chapter/10.1007/978-3-319-92384-0_1
Abstract not found: http://link.springer.com/article/10.1007/s00249-019-01373-4
Abstract not found: http://link.springer.com/article/10.1007/s11036-022-02014-5
Abstract not found: http://link.springer.com/article/10.1007/s12551-021-00845-2
Abstract not found: http://link.springer.com/article/10.1007/s13361-019-02232-4
Abstract not found: http://link.springe

In [ ]:
df_springer2

,Item Title,Publication Title,Book Series Title,Journal Volume,Journal Issue,Item DOI,Authors,Publication Year,URL,Content Type,Abstract,keywords
0,An Efficient Low-Cost Smart Water Monitoring S...,Artificial Intelligence Applications for Smart...,NaN,NaN,NaN,10.1007/978-3-030-63068-3_3,S. Abdul RahamanHaitham A. Jamil,2021,http://link.springer.com/chapter/10.1007/978-3...,Chapter,Water scarcity is a threat to livelihood in th...,Smart water management; IoT-based water monito...
1,The Internet of Things Based Water Quality Mon...,Smart Systems and IoT: Innovations in Computing,NaN,NaN,NaN,10.1007/978-981-13-8406-6_39,Neha DalwadiMamta Padole,2020,http://link.springer.com/chapter/10.1007/978-9...,Chapter,The accessibility to high-quality water is the...,Water quality monitoring; Sensors; pH; Turbidi...
2,A Cost-Effective Real-Time Monitoring System f...,Science and Technologies for Smart Cities,NaN,NaN,NaN,10.1007/978-3-030-51005-3_26,Gonçalo MarquesRui Pitarma,2020,http://link.springer.com/chapter/10.1007/978-3...,Chapter,Water is a limited and essential resource to h...,Agriculture; Enhanced living environments; Int...
3,Internet of Things for Water Quality Monitorin...,Artificial Intelligence for Sustainable Develo...,NaN,NaN,NaN,10.1007/978-3-030-51920-9_13,Joshua O. IghaloAdewale George AdeniyiGoncalo ...,2021,http://link.springer.com/chapter/10.1007/978-3...,Chapter,The implementation of urbanisation and industr...,Actuators; Environment; Internet of things; Se...
4,Internet of Things Concept and Its Applications,Smart Sensors for Industrial Internet of Things,NaN,NaN,NaN,10.1007/978-3-030-52624-5_2,Prashant AhluwaliaNitin Mittal,2021,http://link.springer.com/chapter/10.1007/978-3...,Chapter,In the current era of digital communication an...,Internet of Things; Smart grid; Solid waste ma...
...,...,...,...,...,...,...,...,...,...,...,...,...
885,Recent developments in MnO2-based photocatalys...,Environmental Science and Pollution Research,NaN,27.0,6,10.1007/s11356-019-07568-8,Sin-Ling ChiamSwee-Yong PungFei-Yee Yeoh,2020,http://link.springer.com/article/10.1007/s1135...,Article,The textile industry consumes a large volume o...,Manganese dioxide (MnO2); Photocatalyst; Organ...
886,RETRACTED ARTICLE: Drought Prediction and Anal...,International Journal of Speech Technology,NaN,25.0,3,10.1007/s10772-021-09850-y,J. BalajeeM. A. Saleem Durai,2022,http://link.springer.com/article/10.1007/s1077...,Article,Understanding and researching water level chan...,Deep Learning; Landsat; Normalized Difference ...
887,"Removal of Pb2+, Cr3+ and Hg2+ ions from aqueo...",Journal of Sol-Gel Science and Technology,NaN,103.0,1,10.1007/s10971-022-05830-z,A. F. P. Allwin Mabes RajSara KrajncMaja Bauma...,2022,http://link.springer.com/article/10.1007/s1097...,Article,Herein we present silica (SiO2) and amino-func...,Amino functionalized SiO2 particles; Adsorptio...
888,Development of a three-dimensional geological ...,Hydrogeology Journal,NaN,28.0,3,10.1007/s10040-020-02113-1,Richard FortierDavid-Roy BanvilleRichard Léves...,2020,http://link.springer.com/article/10.1007/s1004...,Article,Among the few positive impacts of climate warm...,Quaternary deposits; Permafrost; Groundwater e...


## Salva em CSV para não precisar realizar o processo de obter o Abstract novamente:

In [ ]:
df_springer2.to_csv(f"{drive_base}BibTeX/Springer Link/springer-com-abstract.csv", index=False)

In [ ]:
df_springer_abstract = pd.read_csv(f"{drive_base}BibTeX/Springer Link/springer-com-abstract.csv")

In [ ]:
df_springer_abstract = df_springer2

## Filtro no Abstract e Titulo

("water quality" AND ("sensor" OR "Sensors") AND (("low cost" OR "low-cost") OR ("Internet of things" OR "IoT")))

In [ ]:
def filter_df(df):
  df['title_abstract'] = df['Item Title'].map(str) + ' ' + df['Abstract'].map(str) + ' ' + df['Keywords'].map(str)


#("water quality" AND ("sensor" OR "Sensors") AND (("low cost" OR "low-cost") OR ("Internet of things" OR "IoT")))

  #alternative
  # pattern = re.compile(r'(?=.*\bwater quality\b)(?=.*(?:(?=.*\bsensor\b)|(?=.*\bSensors\b)))(?=.*(?:(?=.*(?:(?=.*\blow cost\b)|(?=.*\blow-cost\b)))|(?=.*(?:(?=.*\bInternet of things\b)|(?=.*\bIoT\b)))))', re.IGNORECASE)
  
  mask_wq = df['title_abstract'].str.contains("water quality") 
  mask_sensor = df['title_abstract'].str.contains("sensor|sensors")
  mask_lowcost = df['title_abstract'].str.contains("low cost|low-cost")
  mask_iot = df['title_abstract'].str.contains("Internet of things|IoT")
      
  df["filtered"] = mask_wq & mask_sensor & (mask_lowcost | mask_iot)

  df.drop(["title_abstract"], axis=1, inplace=True)
  return df

In [ ]:
df_springer_filtered = filter_df(df_springer_abstract)


In [ ]:
df_springer_filtered.groupby(['filtered']).size()

filtered
False    2801
True      127
dtype: int64

In [ ]:
df_temp = df_springer_filtered[df_springer_filtered['filtered']==True]

In [ ]:
df_temp.to_csv(f"{drive_base}BibTeX/Springer Link/filtrado.csv", sep=",")  